1. Importing all utilities and libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import re

2. Opening the json files containing the segmentation information 500 images (entries) at a time

In [ ]:
f500 = open('/content/drive/MyDrive/DISSERTATION/small_500.json')
data500 = json.load(f500)

f1000 = open('/content/drive/MyDrive/DISSERTATION/small_1000.json')
data1000 = json.load(f1000)

f1500 = open('/content/drive/MyDrive/DISSERTATION/small_1500.json')
data1500 = json.load(f1500)

f2000 = open('/content/drive/MyDrive/DISSERTATION/small_2000.json')
data2000 = json.load(f2000)

f2500 = open('/content/drive/MyDrive/DISSERTATION/small_2500.json')
data2500 = json.load(f2500)

f3000 = open('/content/drive/MyDrive/DISSERTATION/small_3000.json')
data3000 = json.load(f3000)

f3500 = open('/content/drive/MyDrive/DISSERTATION/small_3500.json')
data3500 = json.load(f3500)

f3600 = open('/content/drive/MyDrive/DISSERTATION/small_rest.json')
data3600 = json.load(f3600)


3. From the Image Segmentation results, calculate the follow indexes: Walkability, Green View Index, Motorization and Enclosre

In [ ]:
labels = {0: 'road',
 1: 'sidewalk',
 2: 'building',
 3: 'wall',
 4: 'fence',
 5: 'pole',
 6: 'traffic light',
 7: 'traffic sign',
 8: 'vegetation',
 9: 'terrain',
 10: 'sky',
 11: 'person',
 12: 'rider',
 13: 'car',
 14: 'truck',
 15: 'bus',
 16: 'train',
 17: 'motorcycle',
 18: 'bicycle'}

pattern = r"^(\d+)_(\d+\.\d+)_(\d+\.\d+)_(\d+)\.jpg(\d+)\.jpg$"

walkability = [1,9,11]

motorization = [0,13,14,15,16,17]

greenery = [8,9]

enclosure_numerator = [2,3,4,8]

enclosure_denominator = [0,1,9,10]

def calculate_index(dataset):

  for item in dataset:
    match = re.match(pattern, item['filename'])
    if match:
      item['id'] = match.group(1)
      item['latitude'] = match.group(2)
      item['longitude'] = match.group(3)
      item['angle'] = match.group(5)

    arr = np.array(item['segmentaion'])
    arr = arr.flatten()
    item['segmentaion'] = arr

    unique_numbers, counts = np.unique(arr, return_counts=True)

    walkability_counts = counts[np.isin(unique_numbers, walkability)]
    walkability_sum = np.sum(walkability_counts)
    item['walkability_index'] = walkability_sum / arr.size

    greenery_counts = counts[np.isin(unique_numbers, greenery)]
    greenery_sum = np.sum(greenery_counts)
    item['greenery_index'] = greenery_sum / arr.size

    motorization_counts = counts[np.isin(unique_numbers, motorization)]
    motorization_sum = np.sum(motorization_counts)
    item['motorization_index'] = motorization_sum / arr.size

    enclosure_numer = counts[np.isin(unique_numbers, enclosure_numerator)]
    enclosure_numer_sum = np.sum(enclosure_numer)
    enclosure_deno = counts[np.isin(unique_numbers, enclosure_denominator)]
    enclosure_deno_sum = np.sum(enclosure_deno)
    item['enclosure_index'] = enclosure_numer_sum / enclosure_deno_sum

  selected_keys = ["id", "latitude", "longitude", "angle", "walkability_index", "greenery_index", "motorization_index", "enclosure_index" ]

  filtered_data = [{key: d[key] for key in selected_keys if key in d} for d in data500]

  df = pd.DataFrame(filtered_data)

  return df



In [ ]:
print(data500[250]['filename'])
data500[250]['motorization_index']

1875_4.859959425490000_52.342861646099998_201406.jpg270.jpg


0.8602417695473251

In [ ]:
first500 = calculate_index(data500)
first1000 = calculate_index(data1000)
first1500 = calculate_index(data1500)
first2000 = calculate_index(data2000)
first2500 = calculate_index(data2500)
first3000 = calculate_index(data3000)
first3500 = calculate_index(data3500)
first3600 = calculate_index(data3600)

4. Export the results as a CSV spreadsheet

In [ ]:
final_csv = pd.concat([first500,first1000,first1500,first2000,first2500,first3000,first3500,first3600])
final_csv = final_csv.drop('angle', axis=1)
aggregation_dict = {
    "latitude": "first",
    "longitude": "first",
    "walkability_index":"mean",
    "greenery_index":"mean",
    "motorization_index":"mean",
    "enclosure_index":"mean"
}
final_csv = final_csv.groupby("id", as_index=False).agg(aggregation_dict).reset_index()
final_csv.to_csv("index.csv", index=False)